# Database Indexing: B-tree, Hash, and Composite Indexes

Indexes are critical database structures that dramatically improve query performance by providing efficient access paths to data. This notebook explores the main index types, their use cases, and demonstrates performance differences with practical examples.

## Index Types Overview

### B-tree Index
**Structure:** Self-balancing tree where each node can have multiple children. Maintains sorted data and allows searches, sequential access, insertions, and deletions in O(log n).

**Best for:**
- Range queries (`BETWEEN`, `<`, `>`, `<=`, `>=`)
- Equality comparisons (`=`)
- `ORDER BY` operations
- `LIKE` patterns with prefix matching (`LIKE 'abc%'`)

**Not ideal for:**
- Pattern matching without prefix (`LIKE '%abc'`)
- Very low cardinality columns

---

### Hash Index
**Structure:** Uses a hash function to map keys to buckets. Provides O(1) average lookup time.

**Best for:**
- Exact equality comparisons (`=`)
- High-volume point lookups
- In-memory databases

**Not ideal for:**
- Range queries
- `ORDER BY` operations
- Pattern matching

> **Note:** SQLite doesn't support hash indexes natively. PostgreSQL, MySQL (MEMORY engine), and other databases do.

---

### Composite (Multi-column) Index
**Structure:** B-tree index on multiple columns, ordered left-to-right.

**Best for:**
- Queries filtering on multiple columns together
- Covering indexes (all queried columns in index)

**Key Rule: Leftmost Prefix**
- Index on `(a, b, c)` can be used for:
  - Queries on `a`
  - Queries on `a, b`
  - Queries on `a, b, c`
- **Cannot** be used efficiently for queries on just `b` or `c`

## Setup: Create Sample Database

In [ ]:
import sqlite3
import random
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create in-memory database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create table without indexes
cursor.execute('''
    CREATE TABLE orders (
        id INTEGER PRIMARY KEY,
        customer_id INTEGER,
        product_category TEXT,
        order_date TEXT,
        amount REAL,
        status TEXT
    )
''')

# Generate sample data
categories = ['Electronics', 'Clothing', 'Books', 'Home', 'Sports']
statuses = ['pending', 'shipped', 'delivered', 'cancelled']

print("Inserting 100,000 sample records...")
data = [
    (
        i,
        random.randint(1, 10000),
        random.choice(categories),
        f"2024-{random.randint(1,12):02d}-{random.randint(1,28):02d}",
        round(random.uniform(10, 1000), 2),
        random.choice(statuses)
    )
    for i in range(100000)
]

cursor.executemany('INSERT INTO orders VALUES (?, ?, ?, ?, ?, ?)', data)
conn.commit()

print(f"Created table with {cursor.execute('SELECT COUNT(*) FROM orders').fetchone()[0]:,} records")

## EXPLAIN QUERY PLAN: Understanding Query Execution

`EXPLAIN QUERY PLAN` shows how SQLite will execute a query:
- **SCAN**: Full table scan (slow, reads all rows)
- **SEARCH**: Uses index (fast, reads only matching rows)
- **USING INDEX**: Specifies which index is used

In [ ]:
def explain_query(query, params=()):
    """Display the query execution plan."""
    plan = cursor.execute(f"EXPLAIN QUERY PLAN {query}", params).fetchall()
    print(f"Query: {query}")
    print(f"Parameters: {params}")
    print("Execution Plan:")
    for row in plan:
        print(f"  {row[-1]}")
    print()

# Query WITHOUT index - Full Table Scan
print("=" * 60)
print("BEFORE INDEXES")
print("=" * 60)
explain_query("SELECT * FROM orders WHERE customer_id = ?", (5000,))
explain_query("SELECT * FROM orders WHERE order_date BETWEEN ? AND ?", ('2024-06-01', '2024-06-30'))
explain_query("SELECT * FROM orders WHERE product_category = ? AND status = ?", ('Electronics', 'shipped'))

## Creating Indexes

In [ ]:
# Create B-tree index on customer_id (default index type in SQLite)
cursor.execute('CREATE INDEX idx_customer_id ON orders(customer_id)')

# Create B-tree index on order_date for range queries
cursor.execute('CREATE INDEX idx_order_date ON orders(order_date)')

# Create COMPOSITE index on (product_category, status)
cursor.execute('CREATE INDEX idx_category_status ON orders(product_category, status)')

conn.commit()
print("Indexes created successfully!")

# List all indexes
print("\nIndexes on 'orders' table:")
indexes = cursor.execute("""
    SELECT name, sql FROM sqlite_master 
    WHERE type='index' AND tbl_name='orders'
""").fetchall()

for idx_name, idx_sql in indexes:
    if idx_sql:  # Skip auto-generated primary key index
        print(f"  • {idx_name}: {idx_sql}")

In [ ]:
# Query WITH index - Using Index Search
print("=" * 60)
print("AFTER INDEXES")
print("=" * 60)
explain_query("SELECT * FROM orders WHERE customer_id = ?", (5000,))
explain_query("SELECT * FROM orders WHERE order_date BETWEEN ? AND ?", ('2024-06-01', '2024-06-30'))
explain_query("SELECT * FROM orders WHERE product_category = ? AND status = ?", ('Electronics', 'shipped'))

## Composite Index: Leftmost Prefix Rule

Let's demonstrate how composite indexes work with the leftmost prefix rule.

In [ ]:
print("Composite Index: idx_category_status ON (product_category, status)")
print("=" * 70)

# ✅ Uses index - queries on leftmost column(s)
print("✅ CAN use composite index:")
explain_query("SELECT * FROM orders WHERE product_category = ?", ('Electronics',))
explain_query("SELECT * FROM orders WHERE product_category = ? AND status = ?", ('Electronics', 'shipped'))

# ❌ Cannot use composite index - query on non-leftmost column only
print("❌ CANNOT use composite index (queries on 'status' only):")
explain_query("SELECT * FROM orders WHERE status = ?", ('shipped',))

## Performance Benchmarking

In [ ]:
def benchmark_query(query, params=(), iterations=100):
    """Benchmark query execution time over multiple iterations."""
    times = []
    for _ in range(iterations):
        start = time.perf_counter()
        cursor.execute(query, params).fetchall()
        times.append((time.perf_counter() - start) * 1000)  # Convert to ms
    return sum(times) / len(times)

# Create a fresh connection to test without indexes
conn_no_idx = sqlite3.connect(':memory:')
cur_no_idx = conn_no_idx.cursor()
cur_no_idx.execute('''
    CREATE TABLE orders (
        id INTEGER PRIMARY KEY, customer_id INTEGER, product_category TEXT,
        order_date TEXT, amount REAL, status TEXT
    )
''')
cur_no_idx.executemany('INSERT INTO orders VALUES (?, ?, ?, ?, ?, ?)', data)
conn_no_idx.commit()

# Define test queries
test_cases = [
    ("Point Lookup (customer_id)", "SELECT * FROM orders WHERE customer_id = ?", (5000,)),
    ("Range Query (order_date)", "SELECT * FROM orders WHERE order_date BETWEEN ? AND ?", ('2024-06-01', '2024-06-30')),
    ("Composite (category + status)", "SELECT * FROM orders WHERE product_category = ? AND status = ?", ('Electronics', 'shipped')),
]

results = []
print("Running benchmarks (100 iterations each)...\n")

for name, query, params in test_cases:
    # Benchmark without index
    time_no_idx = benchmark_query(query, params, iterations=100)
    cur_no_idx.execute(query, params)
    
    # Benchmark with index (using original cursor)
    time_with_idx = benchmark_query(query, params, iterations=100)
    
    speedup = time_no_idx / time_with_idx if time_with_idx > 0 else 0
    results.append({
        'query': name,
        'without_index': time_no_idx,
        'with_index': time_with_idx,
        'speedup': speedup
    })
    print(f"{name}:")
    print(f"  Without Index: {time_no_idx:.3f} ms")
    print(f"  With Index:    {time_with_idx:.3f} ms")
    print(f"  Speedup:       {speedup:.1f}x faster\n")

conn_no_idx.close()

## Performance Visualization

In [ ]:
# Create comparison bar chart
queries = [r['query'] for r in results]
times_no_idx = [r['without_index'] for r in results]
times_with_idx = [r['with_index'] for r in results]
speedups = [r['speedup'] for r in results]

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Query Execution Time (ms)', 'Speedup Factor'),
    specs=[[{"type": "bar"}, {"type": "bar"}]]
)

# Execution time comparison
fig.add_trace(
    go.Bar(name='Without Index', x=queries, y=times_no_idx, marker_color='#EF553B'),
    row=1, col=1
)
fig.add_trace(
    go.Bar(name='With Index', x=queries, y=times_with_idx, marker_color='#00CC96'),
    row=1, col=1
)

# Speedup factor
fig.add_trace(
    go.Bar(
        name='Speedup',
        x=queries,
        y=speedups,
        marker_color='#636EFA',
        text=[f'{s:.1f}x' for s in speedups],
        textposition='outside'
    ),
    row=1, col=2
)

fig.update_layout(
    title_text='Database Index Performance Impact',
    height=450,
    showlegend=True,
    barmode='group'
)

fig.update_yaxes(title_text='Time (ms)', row=1, col=1)
fig.update_yaxes(title_text='Speedup (x times faster)', row=1, col=2)

fig

## Index Trade-offs: Write Performance Impact

In [ ]:
def benchmark_writes(cursor_obj, has_indexes=False, iterations=1000):
    """Benchmark INSERT performance."""
    times = []
    for i in range(iterations):
        record = (
            100000 + i,
            random.randint(1, 10000),
            random.choice(categories),
            f"2025-{random.randint(1,12):02d}-{random.randint(1,28):02d}",
            round(random.uniform(10, 1000), 2),
            random.choice(statuses)
        )
        start = time.perf_counter()
        cursor_obj.execute('INSERT INTO orders VALUES (?, ?, ?, ?, ?, ?)', record)
        times.append((time.perf_counter() - start) * 1000)
    return sum(times) / len(times)

# Create fresh databases for write testing
# Without indexes
conn_write_no_idx = sqlite3.connect(':memory:')
cur_write_no_idx = conn_write_no_idx.cursor()
cur_write_no_idx.execute('''
    CREATE TABLE orders (
        id INTEGER PRIMARY KEY, customer_id INTEGER, product_category TEXT,
        order_date TEXT, amount REAL, status TEXT
    )
''')

# With indexes
conn_write_with_idx = sqlite3.connect(':memory:')
cur_write_with_idx = conn_write_with_idx.cursor()
cur_write_with_idx.execute('''
    CREATE TABLE orders (
        id INTEGER PRIMARY KEY, customer_id INTEGER, product_category TEXT,
        order_date TEXT, amount REAL, status TEXT
    )
''')
cur_write_with_idx.execute('CREATE INDEX idx_customer_id ON orders(customer_id)')
cur_write_with_idx.execute('CREATE INDEX idx_order_date ON orders(order_date)')
cur_write_with_idx.execute('CREATE INDEX idx_category_status ON orders(product_category, status)')

print("Benchmarking write performance (1000 INSERTs each)...\n")

write_no_idx = benchmark_writes(cur_write_no_idx, has_indexes=False)
write_with_idx = benchmark_writes(cur_write_with_idx, has_indexes=True)

print(f"INSERT without indexes: {write_no_idx:.4f} ms/record")
print(f"INSERT with 3 indexes:  {write_with_idx:.4f} ms/record")
print(f"Write overhead:         {(write_with_idx/write_no_idx - 1)*100:.1f}% slower")

conn_write_no_idx.close()
conn_write_with_idx.close()

In [ ]:
# Visualize read vs write trade-off
fig = go.Figure()

operations = ['Reads (Point Lookup)', 'Writes (INSERT)']
no_index_perf = [results[0]['without_index'], write_no_idx]
with_index_perf = [results[0]['with_index'], write_with_idx]

fig.add_trace(go.Bar(
    name='Without Index',
    x=operations,
    y=no_index_perf,
    marker_color='#EF553B',
    text=[f'{v:.3f} ms' for v in no_index_perf],
    textposition='outside'
))

fig.add_trace(go.Bar(
    name='With Index',
    x=operations,
    y=with_index_perf,
    marker_color='#00CC96',
    text=[f'{v:.3f} ms' for v in with_index_perf],
    textposition='outside'
))

fig.update_layout(
    title='Index Trade-off: Read vs Write Performance',
    yaxis_title='Time (ms)',
    barmode='group',
    height=400
)

fig

## 📋 Key Takeaways

### When to Use Each Index Type

| Index Type | Use When | Avoid When |
|------------|----------|------------|
| **B-tree** | Range queries, sorting, most general use | Very low cardinality columns |
| **Hash** | Exact equality lookups only (=) | Range queries, ORDER BY |
| **Composite** | Multi-column WHERE clauses | Queries don't use leftmost columns |

### Best Practices

1. **Index columns in WHERE, JOIN, ORDER BY clauses** - These are the most impactful

2. **Use EXPLAIN QUERY PLAN** - Always verify your indexes are being used

3. **Composite index column order matters** - Put highest cardinality / most selective column first

4. **Consider the read/write ratio** - Indexes speed up reads but slow down writes

5. **Avoid over-indexing** - Each index adds storage and write overhead

6. **Monitor and maintain** - Regularly analyze query patterns and rebuild fragmented indexes

### Performance Impact Summary

- **Reads**: Can improve 10-1000x depending on table size and selectivity
- **Writes**: Each index adds ~10-50% overhead per INSERT/UPDATE
- **Storage**: Indexes typically add 10-30% storage overhead per indexed column

In [ ]:
# Cleanup
conn.close()
print("Database connection closed.")